In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pydotplus

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
np.random.seed(1)
import keras
from keras.models import Model
from keras.layers import Activation, Dense, Dropout, Input, LSTM, concatenate
from keras import optimizers
import tensorflow as tf
tf.random.set_seed(1)

In [11]:
#Function to calculate Exponential Moving Average using simple moving average 
def ema(values, i):
    #Simple moving average
    sma = np.mean(values[:, 3])
    ema_values = [sma]
    k = 2 / (1 + i)
    for i in range(len(his) - i, len(his)):
        close = his[i][3]
        ema_values.append(close * k + ema_values[-1] * (1 - k))
    return ema_values[-1]

In [12]:
#history_points represents the set number of days that affect the next
history_points = 50

In [13]:
#Import .csv data and remove the Date attribute 
data = pd.read_csv('SPY.csv')
data = data.drop('Date', axis=1)
data = data.drop('Adj Close', axis=1)
data = data.values

In [14]:
#Normalize the data to a MinMaxScaler
data_normaliser = preprocessing.MinMaxScaler()
data_normalised = data_normaliser.fit_transform(data)

In [15]:
print(data_normalised.shape)

(7075, 5)


In [16]:
#Using history_points, open, close, high, low, volume, data, points create OpenHighLowChart and next day open values
ohlcv_histories_normalised = np.array([data_normalised[i:i + history_points].copy() for i in range(len(data_normalised) - history_points)])
next_day_open_values_normalised = np.array([data_normalised[:, 0][i + history_points].copy() for i in range(len(data_normalised) - history_points)])
next_day_open_values_normalised = np.expand_dims(next_day_open_values_normalised, -1)

In [17]:
next_day_open_values = np.array([data[:, 0][i + history_points].copy() for i in range(len(data) - history_points)])
next_day_open_values = np.expand_dims(next_day_open_values, -1)

In [18]:
#Normalize the data to a MinMaxScaler
y_normaliser = preprocessing.MinMaxScaler()
y_normaliser.fit(next_day_open_values)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [19]:
#Create technical indicators using simple and exponential moving average
technical_indicators = []
for his in ohlcv_histories_normalised:
    #Simple moving average of the closing price
    sma = np.mean(his[:, 3])
    macd = ema(his, 12) - ema(his, 26)
    #Add the simple and exponential moving average to the technical indicator 
    technical_indicators.append(np.array([sma,macd,]))

In [20]:
technical_indicators = np.array(technical_indicators)

tech_ind_scaler = preprocessing.MinMaxScaler()
technical_indicators_normalised = tech_ind_scaler.fit_transform(technical_indicators)

In [21]:
assert ohlcv_histories_normalised.shape[0] == next_day_open_values_normalised.shape[0] == technical_indicators_normalised.shape[0], "data shapes are inconsistent"

In [22]:
#Data Preparation
#Split the data into Training and Testing 9-1 ratio
test_split = 0.9
#Split number (6322)
n = int(ohlcv_histories_normalised.shape[0] * test_split)

In [23]:
print(ohlcv_histories_normalised.shape)
print(technical_indicators_normalised.shape)

(7025, 50, 5)
(7025, 2)


In [24]:
ohlcv_train = ohlcv_histories_normalised[:n]
tech_ind_train = technical_indicators_normalised[:n]
y_train = next_day_open_values_normalised[:n]

In [25]:
ohlcv_test = ohlcv_histories_normalised[n:]
tech_ind_test = technical_indicators_normalised[n:]
y_test = next_day_open_values_normalised[n:]

unscaled_y_test = next_day_open_values[n:]

In [26]:
print(ohlcv_train.shape)
print(ohlcv_test.shape)

(6322, 50, 5)
(703, 50, 5)


In [27]:
#Create LSTM model
#Define two sets of inputs
lstm_input = Input(shape=(history_points, 5), name='lstm_input')
dense_input = Input(shape=(technical_indicators_normalised.shape[1],), name='tech_input')

In [28]:
#First branch operates on the first input
x = LSTM(history_points, name='lstm_0')(lstm_input)
x = Dropout(0.2, name='lstm_dropout_0')(x)
lstm_branch = Model(inputs=lstm_input, outputs=x)

In [29]:
#Second branch opreates on the second input
y = Dense(20, name='tech_dense_0')(dense_input)
y = Activation("relu", name='tech_relu_0')(y)
y = Dropout(0.2, name='tech_dropout_0')(y)
technical_indicators_branch = Model(inputs=dense_input, outputs=y)

In [30]:
#Combine the output of the two branches
combined = concatenate([lstm_branch.output, technical_indicators_branch.output], name='concatenate')

In [31]:
z = Dense(64, activation="sigmoid", name='dense_pooling')(combined)
z = Dense(1, activation="linear", name='dense_out')(z)

In [32]:
#Model will accept the inputs of the two branches and then output a single value
model = Model(inputs=[lstm_branch.input, technical_indicators_branch.input], outputs=z)
adam = optimizers.Adam(lr=0.0005)
model.compile(optimizer=adam, loss='mse')

In [33]:
#Print model diagram
import pydot
import pydotplus
import graphviz
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [ ]:
#Train Model model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 100, batch_size = 64, verbose = 1)
model.fit(x=[ohlcv_train, tech_ind_train], y=y_train, validation_data=([ohlcv_test, tech_ind_test],y_test), batch_size=32, epochs=history_points, shuffle=True, validation_split=0.1)

Epoch 1/50
178/178 [==============================] - 8s 37ms/step - loss: 0.1816 - val_loss: 0.0053
Epoch 2/50
178/178 [==============================] - 6s 33ms/step - loss: 0.0012 - val_loss: 0.0019
Epoch 3/50
178/178 [==============================] - 6s 32ms/step - loss: 8.2286e-04 - val_loss: 1.9965e-04
Epoch 4/50
178/178 [==============================] - 6s 32ms/step - loss: 6.8733e-04 - val_loss: 1.6722e-04
Epoch 5/50
178/178 [==============================] - 6s 33ms/step - loss: 5.9437e-04 - val_loss: 1.6238e-04
Epoch 6/50
178/178 [==============================] - 6s 32ms/step - loss: 5.1363e-04 - val_loss: 5.6247e-04
Epoch 7/50
178/178 [==============================] - 6s 32ms/step - loss: 4.3446e-04 - val_loss: 2.8220e-04
Epoch 8/50
178/178 [==============================] - 6s 32ms/step - loss: 3.9332e-04 - val_loss: 2.0411e-04
Epoch 9/50
178/178 [==============================] - 6s 32ms/step - loss: 3.4700e-04 - val_loss: 8.1840e-05
Epoch 10/50
178/178 [==============

In [ ]:
#Model evaluation
evaluation = model.evaluate([ohlcv_test, tech_ind_test], y_test)
print(evaluation)

In [ ]:
print(ohlcv_test.shape)
print(tech_ind_test.shape)

In [ ]:
#Predict and check model performance 
y_test_predicted = model.predict([ohlcv_test, tech_ind_test])
y_predicted = model.predict([ohlcv_histories_normalised, technical_indicators])

In [ ]:
#Denomaization or scaler inverse
y_test_predicted = y_normaliser.inverse_transform(y_test_predicted)
y_predicted = y_normaliser.inverse_transform(y_predicted)

In [ ]:
assert unscaled_y_test.shape == y_test_predicted.shape, "data shapes are inconsistent"

In [ ]:
#Calculate Root Mean Square Error performance 
rmse = np.mean(np.square(unscaled_y_test - y_test_predicted))
scaled_mse = rmse / (np.max(unscaled_y_test) - np.min(unscaled_y_test)) * 100
print(scaled_mse)

In [ ]:
import matplotlib.pyplot as plt
#Plot
plt.gcf().set_size_inches(18, 9, forward=True)
real = plt.plot(unscaled_y_test[0:-1], label='real')
#Shift test predictions for plotting
pred = plt.plot(y_test_predicted[0:-1], label='predicted')
#Plot baseline and prediction
plt.legend(['Real', 'Predicted'])
plt.show()

In [ ]:
#Save model for trade program
model.save(f'time_series_model.h5')